<a href="https://colab.research.google.com/github/takakishi/HEC_DS_ML_project/blob/main/src/MF_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# spaCy
!pip install -U spacy
!python -m spacy download fr_core_news_md
!python -m spacy download fr_core_news_lg

2023-11-25 13:20:09.962916: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 13:20:09.963031: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 13:20:09.963082: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 13:20:09.983115: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-25 13:20:12.243998: W tensorflow/compiler/

In [55]:
# Libraries
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Word2Vec
import spacy

In [56]:
# Import Data
sample_submission = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/sample_submission.csv')
training_data = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/training_data.csv')
unlabelled_test_data = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/unlabelled_test_data.csv')

In [40]:
display(sample_submission.head())
display(sample_submission.shape)

,id,difficulty
0,0,A1
1,1,A1
2,2,A1
3,3,A1
4,4,A1


(1200, 2)

In [57]:
display(training_data.head())

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


In [58]:
display(unlabelled_test_data.head())
display(unlabelled_test_data.shape)

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."


(1200, 2)

In [59]:
# Define functions

# Functions that :
  # convert to lower case
  # remove puncutation
  # Tokenize
  # remove FRENCH stopwords
  # lemmatization
  # check with spacy_tokenizer() in week 5
def preprocess_text(text):
    ## Convert to lower case
    text = text.lower()

    ## Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    ## Tokenization
    text = sp(text) # spaCy object is used to create a document
    #token_list = [token.text for token in my_doc] # Create list of tokens
    # code above not necessary

    ## Remove stopwords
    spacy_stopwords = spacy.lang.fr.stop_words.STOP_WORDS # Import stopwords from FRENCH language -> list of stopwords
    text = [word.text for word in text if word.is_stop == False] # Filter stopwords

    ## lemmatization
    text = sp(" ".join(text))
    text = [word.lemma_ for word in text]

    ## return text
    text = " ".join(text)
    return text

def preprocess_text_nlp(text):
    ## Convert to lower case
    text = text.lower()

    ## Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    ## Tokenization
    text = nlp(text) # spaCy object is used to create a document
    #token_list = [token.text for token in my_doc] # Create list of tokens
    # code above not necessary

    ## Remove stopwords
    spacy_stopwords = spacy.lang.fr.stop_words.STOP_WORDS # Import stopwords from FRENCH language -> list of stopwords
    text = [word.text for word in text if word.is_stop == False] # Filter stopwords

    ## lemmatization
    text = sp(" ".join(text))
    text = [word.lemma_ for word in text]

    ## return text
    text = " ".join(text)
    return text

In [60]:
# Load French language model
sp = spacy.load('fr_core_news_md')

In [61]:
# Text Preprocessing
# Apply preprocess_text to the sentences
training_data['processed_sentence'] = training_data['sentence'].apply(preprocess_text)

In [62]:
# Training and test set
X = training_data['processed_sentence']
y = training_data['difficulty']

# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
# Feature extraction
# Convert your text data into numerical features that can be used by a machine learning model.
# common techniques -> BOW and TF_IDF

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 1))  # Use a custom analyzer to pass preprocessed tokens directly, -> ,analyzer=lambda x: x
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

X_tfidf = vectorizer.fit_transform(X)

# Bag-of-Words Vectorization with custom analyzer
vectorizer = CountVectorizer(ngram_range=(1, 1))
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

X_bow = vectorizer.fit_transform(X)

In [64]:
# TF-IDF and logistic regression
# Define classifier
#classifier = LogisticRegressionCV(penalty='l2', solver='lbfgs', cv=5, max_iter=1000)#LogisticRegression(solver='lbfgs', max_iter=5000)
classifier = LogisticRegression(solver='lbfgs', max_iter=1000)

classifier.fit(X_train_tfidf,y_train)

# Evaluate model
y_pred = classifier.predict(X_test_tfidf)
## Accuracy
accuracy_tfidf = round(accuracy_score(y_test, y_pred), 4)
print(f'The accuracy using TF-IDF is: {accuracy_tfidf*100:0.2f}%')

The accuracy using TF-IDF is: 38.96%


In [65]:
# BOW and logistic regression
# Define classifier
#classifier = LogisticRegressionCV(penalty='l2', solver='lbfgs', cv=5, max_iter=1000)#LogisticRegression(solver='lbfgs', max_iter=5000)
classifier = LogisticRegression(solver='lbfgs', max_iter=1000)

classifier.fit(X_train_bow,y_train)

# Evaluate model
y_pred = classifier.predict(X_test_bow)
## Accuracy
accuracy_bow = round(accuracy_score(y_test, y_pred), 4)
print(f'The accuracy using BOW is: {accuracy_bow*100:0.2f}%')

The accuracy using BOW is: 38.85%


In [66]:
### test for different model -> BOW
### https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]


CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, X_bow, y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))

cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
display(cv_df)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

,model_name,fold_idx,accuracy
0,RandomForestClassifier,0,0.298958
1,RandomForestClassifier,1,0.284375
2,RandomForestClassifier,2,0.277083
3,RandomForestClassifier,3,0.302083
4,RandomForestClassifier,4,0.296875
5,LinearSVC,0,0.403125
6,LinearSVC,1,0.375000
7,LinearSVC,2,0.368750
8,LinearSVC,3,0.375000
9,LinearSVC,4,0.345833


In [67]:
### test for different model -> TF_IDF
### https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]


CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, X_tfidf, y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))

cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
display(cv_df)

,model_name,fold_idx,accuracy
0,RandomForestClassifier,0,0.332292
1,RandomForestClassifier,1,0.328125
2,RandomForestClassifier,2,0.290625
3,RandomForestClassifier,3,0.316667
4,RandomForestClassifier,4,0.315625
5,LinearSVC,0,0.427083
6,LinearSVC,1,0.396875
7,LinearSVC,2,0.393750
8,LinearSVC,3,0.401042
9,LinearSVC,4,0.394792


In [ ]:
### with natural language processing

In [68]:
sample_submission = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/sample_submission.csv')
training_data = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/training_data.csv')
unlabelled_test_data = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/unlabelled_test_data.csv')

In [69]:
### try with word embedding
nlp = spacy.load("fr_core_news_lg")

In [70]:
# Text Preprocessing
# Apply preprocess_text to the sentences
training_data['processed_sentence'] = training_data['sentence'].apply(preprocess_text_nlp)

In [71]:
# Training and test set
X = training_data['processed_sentence']
y = training_data['difficulty']

# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [72]:
# Feature extraction
# Convert your text data into numerical features that can be used by a machine learning model.
# common techniques -> BOW and TF_IDF

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 1))  # Use a custom analyzer to pass preprocessed tokens directly, -> ,analyzer=lambda x: x
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

X_tfidf = vectorizer.fit_transform(X)

# Bag-of-Words Vectorization with custom analyzer
vectorizer = CountVectorizer(ngram_range=(1, 1))
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

X_bow = vectorizer.fit_transform(X)

In [73]:
### test for different model -> BOW
### https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]


CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, X_bow, y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))

cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
display(cv_df)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

,model_name,fold_idx,accuracy
0,RandomForestClassifier,0,0.298958
1,RandomForestClassifier,1,0.284375
2,RandomForestClassifier,2,0.277083
3,RandomForestClassifier,3,0.302083
4,RandomForestClassifier,4,0.296875
5,LinearSVC,0,0.403125
6,LinearSVC,1,0.375000
7,LinearSVC,2,0.368750
8,LinearSVC,3,0.375000
9,LinearSVC,4,0.345833


In [74]:
### test for different model -> TF_IDF
### https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]


CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, X_tfidf, y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))

cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
display(cv_df)

,model_name,fold_idx,accuracy
0,RandomForestClassifier,0,0.332292
1,RandomForestClassifier,1,0.328125
2,RandomForestClassifier,2,0.290625
3,RandomForestClassifier,3,0.316667
4,RandomForestClassifier,4,0.315625
5,LinearSVC,0,0.427083
6,LinearSVC,1,0.396875
7,LinearSVC,2,0.393750
8,LinearSVC,3,0.401042
9,LinearSVC,4,0.394792


In [ ]:
### use Word2Vec, GloVe instead of TF-IDF

In [43]:
sample_submission = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/sample_submission.csv')
training_data = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/training_data.csv')
unlabelled_test_data = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/unlabelled_test_data.csv')

In [44]:
### try with word embedding
nlp = spacy.load("fr_core_news_lg")

In [45]:
# Text Preprocessing
# Apply preprocess_text to the sentences
training_data['processed_sentence'] = training_data['sentence'].apply(preprocess_text_nlp)

In [46]:
# Training and test set
X = training_data['processed_sentence']
y = training_data['difficulty']

# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
# Feature extraction
# Convert your text data into numerical features that can be used by a machine learning model.
# common techniques -> Word2Vec, GloVe

# Word2Vec
# Train Word2Vec model
model = Word2Vec(sentences=X, vector_size=100, window=5, min_count=1, workers=4)

#model.save("word2vec.model")
#model = Word2Vec.load("word2vec.model")




# Convert each document to a vector
X_vec = [document_vector(model, doc) for doc in X]

In [51]:
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]


CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, X_vec, y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))

cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
display(cv_df)

ValueError: ignored